In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import datetime
import matplotlib.pyplot as plt
import DeepFM_v2 as dfm
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import io
import os
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
profile=pd.read_csv('profile.csv')
train=profile.loc[:,['label','id','item_tag']]
train=train.dropna()
y=train.pop('label')
train=train.to_dict(orient='list')
for key in train.keys():
    train[key]=np.array(train[key])

In [ ]:
model=

In [6]:
tag_weights=model.get_layer('tag_embedding').get_weights()[0]
id_weights=model.get_layer('id_embedding').get_weights()[0]
vocab=id_vectorize_layer.get_vocabulary()
out_v=io.open('vectors.tsv','w',encoding='utf-8')
out_m=io.open('metadata.tsv','w',encoding='utf-8')
for index,word in enumerate(vocab):
    if index==0:continue
    vec=id_weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()